Verificacion de archivo para carga incremental de la tabla review

Preparacion para prueba:
Se utilizara el archivo 'review.json' para sacar tres conjuntos de dataset:
    1. 'reviews_inicial.json'     , Archivo con la mayoria de la data chunk 14/17     
    2. 'reviews_incremental.json' , Archivo con datos en perfecto estado chunk 2/17
    3. 'reviews_incremental_con_fallas_columnas.json', Archivo .json con columnas aleatorio
    4. 'reviews_incremental_con fallas_formato_datos.json', Archivo .json con columnas que coinciden 
        con 'reviews_inicial.json' pero su contenido no.
    5. 'reviews_incremental_con_repeticiones.json' , archivo con datos chunk 1/17 y con datos 
        repetidos de chunks anteriores.
nota: no es necesario tomar en cuenta la fecha del review para clasificar
los datos, es preferible datos aleatorios, los chunks que se crean 
cuando se crea  'reviews_inicial.parquet.gzip'

Procedimiento de verificacion (Bussnes Rules):
Al introducir un nuevo archivo .json se verifica en formato:
    1. cantidad de columnas
    2. nombre de columnas
    3. formato de datos en columnas
Si cumple con formato se verifica contenido, se trabaja con los codigos alfanumericos 
del nuevo dato/review review_id,user_id y business_id:

    4. tabla 'idreview[id_review(int),review_id(tex)]';
        4.1. si el review_id ya existen en tabla base 
            se descarta el dato/review nuevo
        4.2. de lo contrario
            se revisan los puntos 5.1 y 6.1
            se incorpora el nuevo review a la tabla reviews y a la tabla idreviews

    5. tabla idusers[id_user(int),user_id(tex)], 
        5.1 si el user_id NO esta en la tabla 'idusers' 
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo user a la tabla 'users' para lo cual se necesitarian los datos [review_count, yelping_since, .... average_stars], al actualizar esa data automatiacamente se actualiza la tabla 'idusers' y despues se procede con el punto 4.2


    6. tabla 'idbusiness[id_business(int),business_id(tex)], 
        6.1 si el business_id del nuevo data/review NO esta en la tabla 'idbusiness'
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo busines a la tabla 'business' para lo cual se necesitarian los datos [name, address, .... categories_id], adicionalmente se debe verifiar su categoria,
    si coincide con las categorias a usar se agrega a la tabla  'business', y automaticamente actualizar las tablas 'idbusiness', 'business_attributes', 'business_categories', 'business_city_state', 'business_hours' para luego proceder con el punto 4.2


Reqerimientos:
    1. Se requiere acceder a la tabla idreviews 
    2. Se requiere  acceder a la tabla idbussines 
    3. Se requiere  acceder a la tabla idusers


 

In [ ]:
#Creacion de datos de prueba
path_incial = 'C:\\Github\\DataSet_YELP\\'
path_final = 'C:\\Github\\DataSet_YELP\\pruebas_incremental\\'
